# Analysing single cell data to strain data

In [1]:
import os
import re
import pandas as pd

import plotly.express as px
from plotly.subplots import make_subplots

## Data wrangling
### wrangle function:
This function transforms the raw data files into a dataframe. <br>
**Input:**<br> 
 - f_dir - directory of raw data files (.txt filesthat contain parameters and features for each cell in the experiment) <br> 
 - strain_data_dir - directory of .xlsx file, contain information about the strain in each well

**Output:** <br>
- output_df - DataFrame containing all the data from the data files in dir<br>

In [2]:
def wrangle(f_dir, strain_data_dir):
    
    files = os.listdir(f_dir)
    
    df_list = [None] * len(files)
    for f in files:
        f_num = f.split('_')[1]
        df = pd.read_csv(os.path.join(f_dir, f), sep='\t')  # load file into dataframe
        # add 'Well ID' column, where the values are plate-row-column indicator for each cell
        df.insert(loc=9, column='Well ID', 
                  value=df['Well Description '].apply(lambda s: f_num + s))  # add plate number columns (file number)
        # remove cells out of gate (where 'R01'==0)
        df = df[df['R01'] == 1]
        # remove unneeded columns
        cols = ['Object ID', 'Well ID', 'Area ', 'Circularity Factor ', 
                'Total Intensity 488nm', 'Mean Intensity 488nm', 
                'Total Intensity 561nm', 'Mean Intensity 561nm', 
               'Center X ', 'Center Y ']
        df = df[cols]
        # remove wells that have less then 30 cells
        df.groupby('Well ID')
        # add df to df_list
        df_list[int(f_num)-1] = df
        print(f_num)

    # concat all dataframes in df_list to one df
    output_df = pd.concat(df_list)
    
    # add strain data to output_df
    strain_df = pd.read_excel(strain_data_dir, usecols=['ORF', 'Gene', '384 Plate_Row_Col'])
    output_df = output_df.merge(right=strain_df, how='left', left_on='Well ID', right_on='384 Plate_Row_Col')
    output_df.drop(columns=['384 Plate_Row_Col'], inplace=True)
    
    return output_df


In [3]:
raw_data_dir = r'D:\user_data\hadar\SMS120 Javier screen 090123\Analysis\ParameterData'
strain_data = '.\List of genes 210 x Tef Cherry oex Javier.xlsx'

df = wrangle(raw_data_dir, strain_data)

10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9


In [4]:
df.head()

,Object ID,Well ID,Area,Circularity Factor,Total Intensity 488nm,Mean Intensity 488nm,Total Intensity 561nm,Mean Intensity 561nm,Center X,Center Y,ORF,Gene
0,0,1A1,1370,1.026662,226123.00,165.05328,145751.0,106.38759,235.0,21.5,YAL002W,VPS8
1,1,1A1,1042,1.088675,230934.00,221.62572,113834.0,109.24568,508.0,22.5,YAL002W,VPS8
2,2,1A1,1237,1.027111,172509.02,139.45757,130191.0,105.24738,187.0,22.5,YAL002W,VPS8
3,3,1A1,1414,1.026263,294291.00,208.12659,154749.0,109.44060,1356.0,25.0,YAL002W,VPS8
4,4,1A1,1283,1.013327,286544.00,223.33905,149158.0,116.25721,1546.5,26.0,YAL002W,VPS8


Remove rows with unwanted ORF/Gene values:

In [5]:
# remove rows from dataframe 'df' columns 'c', where the string value contain a substring

# INPUT:
#      - df - the target dataframe to remove rows from
#      - columns - target columns in df (datatype in those columns needs to be string). list of strings (column names).
#      - substr - sub-string to look for in the string values in df[columns]. list of strings
# OUTPUT:
#      - df - the input dataframe, without rows that have substring from substr in the value of column from columns

def remove_values(df, cols, substr=['blank', 'BLANK', 'Blank', 'None']):
    
    assert all(pd.api.types.is_string_dtype(df[c]) for c in cols), "One of the column in 'columns' has non-string values"
    assert all(isinstance(s, str) for s in substr), "Not all values in 'substr' are 'string'"
    
    for c in cols:
        df = df.loc[~df[c].str.contains(re.compile("|".join(substr))).astype('bool'), :]
    return df

In [6]:
# remove ORF/Gene that have substring from deafault list ['blank', 'BLANK', 'Blank', 'None']

df = remove_values(df, cols=['ORF', 'Gene'])
df.head()

,Object ID,Well ID,Area,Circularity Factor,Total Intensity 488nm,Mean Intensity 488nm,Total Intensity 561nm,Mean Intensity 561nm,Center X,Center Y,ORF,Gene
0,0,1A1,1370,1.026662,226123.00,165.05328,145751.0,106.38759,235.0,21.5,YAL002W,VPS8
1,1,1A1,1042,1.088675,230934.00,221.62572,113834.0,109.24568,508.0,22.5,YAL002W,VPS8
2,2,1A1,1237,1.027111,172509.02,139.45757,130191.0,105.24738,187.0,22.5,YAL002W,VPS8
3,3,1A1,1414,1.026263,294291.00,208.12659,154749.0,109.44060,1356.0,25.0,YAL002W,VPS8
4,4,1A1,1283,1.013327,286544.00,223.33905,149158.0,116.25721,1546.5,26.0,YAL002W,VPS8


Average the dataframe by Gene and ORF columns: <br>
 - add 'Cell Count' column - number of cells/rows that were averaged

In [7]:
# get mean of dataframe groups

# INPUT:
#      - df - the target dataframe
#      - groupby - list of strings, columns names to group df by (Groups in df to calculate the mean of).
# OUTPUT:
#      - avg_df - the average values of each group.
#                 * Has 'Cell Count' column that contain the number of cells (df rows) in each group
#                   (number of items that were used for averaging)
#                 * If 'Cell Count' <= 50 -> remove from avg_df

def avg_df(df, groupby):
    # create dictionary of column:function to aggregate by
    d = dict.fromkeys(df.drop(columns=groupby).columns, 'mean')
    d[groupby[0]] = 'count'
    
    # group and aggregate
    avg_df = df.groupby(groupby).aggregate(d)
    avg_df.rename(columns={groupby[0]:'Cell Count'}, inplace=True)
    
    # remove rows with less then 50 cells and reset index
    avg_df = avg_df[avg_df['Cell Count'] > 50]
    avg_df.reset_index(inplace=True)
    
    return avg_df


In [8]:
cols = ['ORF', 'Gene', 
        'Total Intensity 488nm', 'Mean Intensity 488nm', 
        'Total Intensity 561nm', 'Mean Intensity 561nm']

avg_df = avg_df(df[cols] ,groupby=['ORF', 'Gene'])

avg_df.head()

,ORF,Gene,Total Intensity 488nm,Mean Intensity 488nm,Total Intensity 561nm,Mean Intensity 561nm,Cell Count
0,YAL001C,TFC3,257676.543439,247.428265,108431.296456,104.279925,2412
1,YAL002W,VPS8,236296.242189,210.705175,123860.983017,111.655761,3290
2,YAL003W,EFB1,142514.137255,171.092262,105159.696137,126.232299,102
3,YAL005C,SSA1,234419.365567,207.804484,202299.113358,177.885477,3036
4,YAL007C,ERP2,241148.965770,217.450270,131362.623734,118.245369,4350


In [27]:
# sort values by intensities
avg_df_sorted_488 = avg_df.sort_values(by='Mean Intensity 488nm')
avg_df_sorted_561 = avg_df.sort_values(by='Mean Intensity 561nm')

## Visualize strain data for comparison

In [34]:
fig = px.bar(avg_df_sorted_488, y="ORF", x='Mean Intensity 488nm', title='Mean Intensity 488nm per ORF')
fig.update_layout(autosize=False, width=1000, height=1500)

fig.show();

In [36]:
fig = px.bar(avg_df_sorted_561, y="ORF", x='Mean Intensity 561nm', title='Mean Intensity 561nm per ORF')
fig.update_layout(autosize=False, width=1000, height=1500)

fig.show();

In [37]:
fig = px.bar(avg_df_sorted_488, y="Gene", x='Mean Intensity 488nm', title='Mean Intensity 488nm per Gene')
fig.update_layout(autosize=False, width=1000, height=1500)

fig.show();

In [38]:
fig = px.bar(avg_df_sorted_561, y="Gene", x='Mean Intensity 561nm', title='Mean Intensity 561nm per Gene')
fig.update_layout(autosize=False, width=1000, height=1500)

fig.show();

## Export data

In [41]:
avg_df_sorted_488.to_csv('avg df - sorted by mean intensity 488nm.txt', sep='\t', index=False)
avg_df_sorted_561.to_csv('avg df - sorted by mean intensity 561nm.txt', sep='\t', index=False)